In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image Generation with Imagen on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/image_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/image_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/image_generation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

[Imagen on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview) brings Google's state of the art generative AI capabilities to application developers. With Imagen on Vertex AI, application developers can build next-generation AI products that transform their user's imagination into high quality visual assets, in seconds.

With Imagen, you can do the following:
- Generate novel images using only a text prompt (text-to-image generation).
- Edit an entire uploaded or generated image with a text prompt.
- Edit only parts of an uploaded or generated image using a mask area you define.
- Upscale existing, generated, or edited images.
- Fine-tune a model with a specific subject (for example, a specific handbag or shoe) for image generation.
- Get text descriptions of images with visual captioning.
- Get answers to a question about an image with Visual Question Answering (VQA).

This notebook focuses on **image generation** only. You can read more about image generation feature from Imagen [here](https://cloud.google.com/vertex-ai/docs/generative-ai/image/generate-images).


### Objectives

In this notebook, you will be exploring the image generation features of Imagen using the Vertex AI Python SDK. You will

- generate images using text prompts
- experiment with different parameters, such as:
    - increasing the number of images to be generated
    - fixing a seed number for reproducibility
    - influencing the output images using negative prompts

### Costs

- This notebook uses billable components of Google Cloud:
  - Vertex AI (Imagen)

- Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK for Python

In [ ]:
! pip install --quiet --upgrade --user google-cloud-aiplatform

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, it is recommended to restart the runtime. Run the following cell to restart the current kernel.

The restart process might take a minute or so.

In [ ]:
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

After the restart is complete, continue to the next step.



<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [ ]:
import sys

# Addtional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Define Google Cloud project information (Colab only)

If you are running this notebook on Google Colab, specify the Google Cloud project information to use. In the following cell, you specify your project information, import the Vertex AI package, and initialize the package. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = "<project_id>"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Initialize Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)

### Load the image generation model

The model names from Vertex AI Imagen have two components: model name and version number. The naming convention follow this format: `<model-name>@<version-number>`. For example, `imagegeneration@001` represent the version **001** of the **imagetext** model.



In [ ]:
from vertexai.preview.vision_models import ImageGenerationModel

generation_model = ImageGenerationModel.from_pretrained("imagegeneration@002")

### Generate an image

The `generate_image` function can be used to generate images. All you need is a text prompt.

###  Explore different parameters

The `generate_images` function accepts additional parameters that can be used to influence the generated images. The following sections will explore how to influence the output images through the use of those additional parameters.

In [ ]:
import math
import matplotlib.pyplot as plt


# An axuillary function to display images in grid
def display_images_in_grid(images):
    """Displays the provided images in a grid format. 4 images per row.

    Args:
        images: A list of PIL Image objects representing the images to display.
    """

    # Determine the number of rows and columns for the grid layout.
    nrows = math.ceil(len(images) / 4)  # Display at most 4 images per row
    ncols = min(len(images) + 1, 4)  # Adjust columns based on the number of images

    # Create a figure and axes for the grid layout.
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, 6))

    for i, ax in enumerate(axes.flat):
        if i < len(images):
            # Display the image in the current axis.
            ax.imshow(images[i]._pil_image)

            # Adjust the axis aspect ratio to maintain image proportions.
            ax.set_aspect("equal")

            # Disable axis ticks for a cleaner appearance.
            ax.set_xticks([])
            ax.set_yticks([])
        else:
            # Hide empty subplots to avoid displaying blank axes.
            ax.axis("off")

    # Adjust the layout to minimize whitespace between subplots.
    plt.tight_layout()

    # Display the figure with the arranged images.
    plt.show()

In [ ]:
prompt = "hyperrealistic picture of: tow truck is towing a car that has fallen off the road. use 4k resolution." # @param {"type": "string"}
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    seed=42
)

display_images_in_grid(response.images)

In [ ]:
response.images[0].show()

In [ ]:
prompt = "hyperrealistic picture of: tow truck is towing a car that has fallen off the road. use 4k resolution." # @param {"type": "string"}
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    seed=42
)

display_images_in_grid(response.images)

In [ ]:
original_image_1024_1024 = response.images[0]
original_image_1024_1024.show()


In [ ]:

original_image_2048_2048 = generation_model.upscale_image(original_image_1024_1024, new_size=2048)
original_image_2048_2048.show()

In [ ]:
original_image_4096_4096 = generation_model.upscale_image(original_image_1024_1024, new_size=4096)
original_image_4096_4096.show()

In [ ]:
prompt = "You are the creative assistant working for a digital agency. You need an asset for the car insurance campaign. Your target audience is people of 20-40 years old, located in Madrid, Spain, looking for car insurance. Style: hyperrealistic, 4K. Description: tow truck is towing a car that has fallen off the road." # @param {"type": "string"}
generation_model = ImageGenerationModel.from_pretrained("imagegeneration@005")
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    seed=42
)

display_images_in_grid(response.images)

In [ ]:
prompt = "You are the creative assistant working for a digital agency. You need an asset for the car insurance campaign. Your target audience is people 70 years old, located in Madrid, Spain, looking for car insurance. Style: hyperrealistic, 4K. Description: tow truck is towing a car that has fallen off the road." # @param {"type": "string"}
generation_model = ImageGenerationModel.from_pretrained("imagegeneration@005")
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    seed=42
)

display_images_in_grid(response.images)

In [ ]:
prompt = "Style: hyperrealistic, 4K. Description: tow truck is towing a car that has fallen off the road." # @param {"type": "string"}
generation_model = ImageGenerationModel.from_pretrained("imagegeneration@005")
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    seed=42
)

display_images_in_grid(response.images)

In [ ]:
! pip install pillow

In [ ]:
def crop_image(image, new_width, new_height):
    """Crops an image to a specified width and height.

    Args:
        image (Image): The PIL Image object to crop.
        new_width (int): The desired width of the cropped image.
        new_height (int): The desired height of the cropped image.

    Returns:
        Image: The cropped PIL Image object.
    """

    original_width, original_height = image.size

    # Calculate where to start cropping (to center the image)
    left = (original_width - new_width) // 2
    top = (original_height - new_height) // 2
    right = left + new_width
    bottom = top + new_height

    return image.crop((left, top, right, bottom))


In [ ]:
import random
from PIL import Image

prompt = "hyperrealistic picture of: tow truck is towing a car that has fallen off the road. use 4k resolution." # @param {"type": "string"}
response = generation_model.generate_images(
    prompt=prompt,
    number_of_images=4,
    seed=42
)

# Cropping parameters
desired_width = 500
desired_height = 500

# Iterate through generated images and crop
cropped_images = []
for img in response.images:
    cropped_img = crop_image(img, desired_width, desired_height)
    cropped_images.append(cropped_img)

display_images_in_grid(cropped_images)


## Conclusion

You have explored the Imagen's image generation features through the Vertex AI Python SDK, including the additional parameters that influence image generation. The next step is to enhance your skills by exploring this [prompting guide](https://cloud.google.com/vertex-ai/docs/generative-ai/image/img-gen-prompt-guide?_ga=2.128324367.-2094800479.1701746552&_gac=1.219926379.1701161688.CjwKCAiAvJarBhA1EiwAGgZl0LFQUFOFZUxfNPlzjB4T00PDiLeCIEYfY-coLbX9eUfHKr_i8VbtSBoCEJQQAvD_BwE). Through practice, you will become proficient in the art of image prompting.